# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [4]:
# Import required libraries
import pandas as pd

# Load the DataFrame from the provided URL
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

# Display the results
print(df)


       Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
0               0  DK49336     Arizona              4809.216960       No   
1               1  KX64629  California              2228.525238       No   
2               2  LZ68649  Washington             14947.917300       No   
3               3  XL78013      Oregon             22332.439460      Yes   
4               4  QA50777      Oregon              9025.067525       No   
...           ...      ...         ...                      ...      ...   
10905       10905  FE99816      Nevada             15563.369440       No   
10906       10906  KX53892      Oregon              5259.444853       No   
10907       10907  TL39050     Arizona             23893.304100       No   
10908       10908  WA60547  California             11971.977650       No   
10909       10909  IV32877         NaN              6857.519928      NaN   

       Coverage Education Effective To Date EmploymentStatus Gender  ...  \
0         B

In [8]:
# Display the first few rows of the DataFrame to understand its structure
df.head()

# Display the column names to see what data is available
print(df.columns)

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [13]:
# Calculate the total claim amount
total_claim_amount = df['Total Claim Amount'].sum()

# Display the total claim amount
print(f'Total Claim Amount: ${total_claim_amount}')

Total Claim Amount: $4744631.677058


In [14]:
# Filter the DataFrame
filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]

# Display the filtered DataFrame
filtered_df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [20]:
# Filter the DataFrame for customers who responded "Yes" to the last marketing campaign
filtered_df = df[df['Response'] == 'Yes']

# Group the data by policy type and gender, then calculate the average total claim amount
avg_claim_amount = filtered_df.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean().reset_index().sort_values(by='Total Claim Amount', ascending=False)

# Display the results
print(avg_claim_amount)

      Policy Type Gender  Total Claim Amount
3   Personal Auto      M          457.010178
4    Special Auto      F          453.280164
2   Personal Auto      F          452.965929
0  Corporate Auto      F          433.738499
5    Special Auto      M          429.527942
1  Corporate Auto      M          408.582459


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [27]:
# Group data by state and count the number of customers in each state
state_counts = df.groupby('State').size().reset_index(name='customer_count')
# Set pandas options to display all rows
pd.set_option('display.max_rows', None)
print(state_counts)

        State  customer_count
0     Arizona            1937
1  California            3552
2      Nevada             993
3      Oregon            2909
4  Washington             888


In [24]:
# Filter the results to only include states where there are more than 500 customers
filtered_states = state_counts[state_counts['customer_count'] > 500]

# Display the results
print(filtered_states)

        State  customer_count
0     Arizona            1937
1  California            3552
2      Nevada             993
3      Oregon            2909
4  Washington             888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [29]:
# Group data by education level and gender, then calculate max, min, and median customer lifetime value
agg_results = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).reset_index()

# Display the results
print(agg_results)

              Education Gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [30]:
# Assuming there is a 'Effective To Date' column to extract the month from
# Convert 'Effective To Date' to datetime
df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])

# Extract the month from 'Effective To Date'
df['Month'] = df['Effective To Date'].dt.month

# Group data by state and month, then count the number of policies sold
policy_counts = df.groupby(['State', 'Month']).size().reset_index(name='policy_count')

# Pivot the data to arrange months as columns and states as rows
pivot_table = policy_counts.pivot(index='State', columns='Month', values='policy_count').fillna(0)

# Display the pivoted table
print(pivot_table)

Month          1     2
State                 
Arizona     1008   929
California  1918  1634
Nevada       551   442
Oregon      1565  1344
Washington   463   425


/tmp/ipykernel_71751/1346741624.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [31]:
# Sort the data by the count of policies sold in descending order
sorted_policy_counts = policy_counts.sort_values(by='policy_count', ascending=False)

# Select the top 3 states with the highest number of policies sold
top_states = sorted_policy_counts.groupby('State')['policy_count'].sum().nlargest(3).index

# Filter the data to include only the top 3 states
top_states_data = policy_counts[policy_counts['State'].isin(top_states)]

# Pivot the data to arrange months as columns and states as rows
pivot_table = top_states_data.pivot(index='State', columns='Month', values='policy_count').fillna(0)

# Display the new DataFrame
print(pivot_table)

Month          1     2
State                 
Arizona     1008   929
California  1918  1634
Oregon      1565  1344


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [41]:
# Display the column names to identify the correct columns
print(df.columns)

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type', 'Month'],
      dtype='object')


In [39]:
# Display the unique values in the 'Renew Offer Type' column
print(df['Renew Offer Type'].unique())

['Offer3' 'Offer4' 'Offer2' 'Offer1']


In [47]:
# Filter the DataFrame to get customers who responded "Yes"
responded_df = df[df['Response'] == 'Yes']

# Group the data by offer type and calculate the response count
response_rate = responded_df.groupby('Renew Offer Type').size().reset_index(name='response_count')

# Calculate the total count for each offer type
total_counts = df.groupby('Renew Offer Type').size().reset_index(name='total_count')
response_rate = response_rate.merge(total_counts, on='Renew Offer Type')
response_rate['response_rate'] = response_rate['response_count'] / response_rate['total_count']

# Display the results
print(response_rate[['Renew Offer Type', 'response_rate']])

  Renew Offer Type  response_rate
0           Offer1       0.148338
1           Offer2       0.220344
2           Offer3       0.018476


External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

### Explicación Paso a Paso

1. **Filtrar el DataFrame para obtener los clientes que respondieron "Yes"**:
    ```python
    responded_df = df[df['Response'] == 'Yes']
    ```
    - Este paso filtra el DataFrame original [`df`] para incluir solo las filas donde la columna `'Response'` es igual a `'Yes'`.
    - El resultado es un nuevo DataFrame [`responded_df`] que contiene solo los clientes que respondieron afirmativamente a la campaña de marketing.

2. **Agrupar los datos por tipo de oferta y calcular el conteo de respuestas**:
    ```python
    response_rate = responded_df.groupby('Renew Offer Type').size().reset_index(name='response_count')
    ```
    - Este paso agrupa los datos en [`responded_df`] por la columna `'Renew Offer Type'`.
    - Luego, cuenta el número de respuestas en cada grupo usando [`size()`].
    - El resultado es un DataFrame [`response_rate`] con dos columnas: `'Renew Offer Type'` y `'response_count'`, donde `'response_count'` es el número de respuestas afirmativas para cada tipo de oferta.

3. **Calcular el conteo total para cada tipo de oferta**:
    ```python
    total_counts = df.groupby('Renew Offer Type').size().reset_index(name='total_count')
    ```
    - Este paso agrupa los datos en el DataFrame original [`df`]por la columna `'Renew Offer Type'`.
    - Luego, cuenta el número total de clientes en cada grupo usando [`size()`].
    - El resultado es un DataFrame [`total_counts`] con dos columnas: `'Renew Offer Type'` y `'total_count'`, donde `'total_count'` es el número total de clientes para cada tipo de oferta.

4. **Combinar los conteos de respuestas y los conteos totales**:
    ```python
    response_rate = response_rate.merge(total_counts, on='Renew Offer Type')
    ```
    - Este paso combina los DataFrames [`response_rate`] en base a la columna `'Renew Offer Type'`.
    - El resultado es un DataFrame [`response_rate`] que ahora contiene tres columnas: `'Renew Offer Type'`, `'response_count'`, y `'total_count'`.

5. **Calcular la tasa de respuesta**:
    ```python
    response_rate['response_rate'] = response_rate['response_count'] / response_rate['total_count']
    ```
    - Este paso calcula la tasa de respuesta dividiendo `'response_count'` por `'total_count'` para cada tipo de oferta.
    - El resultado es una nueva columna `'response_rate'` en el DataFrame [`response_rate`] que contiene la tasa de respuesta para cada tipo de oferta.

6. **Mostrar los resultados**:
    ```python
    print(response_rate[['Renew Offer Type', 'response_rate']])
    ```
    - Este paso imprime las columnas `'Renew Offer Type'` y `'response_rate'` del DataFrame [`response_rate`].
    - El resultado es una tabla que muestra la tasa de respuesta para cada tipo de oferta.

Este código permite analizar la efectividad de diferentes tipos de ofertas en términos de la tasa de respuesta de los clientes.